# 

# PART 1: SQL

There are two tables in the included database

1. train_table
2. test_table


IMPORTANT: No other libraries are allowed to solve this tests only SQL Queries allowed

* pandas methods are not is not allowed
* sqlalchemy is not allowed

In [ ]:
# only library allowed do not modify
import pandas as pd
from src.sql import execute_query

In [ ]:
#example query inside docstring
execute_query("""
        SELECT * 
        FROM train_table
""")

In [ ]:
#example query inside docstring
execute_query("""
        SELECT * 
        FROM test_table
""")

## question 1: Basic SQL
Write a SQL statement from table <i>train_table</i> to obtain the top 10 <i>locations</i> people twit from (in descending order)

In [ ]:
query_1 = """
SELECT
    UserName, 
    Location,
    COUNT(*) AS tweet_count
FROM train_table
WHERE Location IS NOT NULL 
GROUP BY Location
ORDER BY tweet_count DESC
LIMIT 10;
"""

In [ ]:
result_1 = execute_query(query_1)

In [ ]:
result_1

## Question 2: Intermediate SQL

In [ ]:
query_2 = """
SELECT
    UserName,
    SUM(CASE WHEN Sentiment = 'Positive' THEN 1 ELSE 0 END) AS pos_count,
    SUM(CASE WHEN Sentiment = 'Negative' THEN 1 ELSE 0 END) AS neg_count,
    SUM(CASE WHEN Sentiment = 'Extremely Positive' THEN 1 ELSE 0 END) AS extremely_pos_count,
    SUM(CASE WHEN Sentiment = 'Extremely Negative' THEN 1 ELSE 0 END) AS extremely_neg_count
FROM train_table
GROUP BY UserName;
"""

In [ ]:
result_2 = execute_query(query_2)

In [ ]:
result_2

## Question 3: Medium SQL

In [ ]:
query_3 = """
--(count of positives + count of extremely positives) / [(count of negatives + count of extremely negatives) + (count of positives + count of extremely positives)]
SELECT
    TweetAt,
    (
        (
            SUM(CASE WHEN Sentiment = 'Positive' THEN 1 ELSE 0 END)
            + SUM(CASE WHEN Sentiment = 'Extremely Positive' THEN 1 ELSE 0 END)
        ) * 1.0
            /
        (
            SUM(CASE WHEN Sentiment IN ('Positive', 'Extremely Positive') THEN 1 ELSE 0 END)
            + SUM(CASE WHEN Sentiment IN ('Negative', 'Extremely Negative') THEN 1 ELSE 0 END)
        )
    ) AS pos_rate
FROM train_table
GROUP BY TweetAt
ORDER BY TweetAt;
"""

In [ ]:
result_3 = execute_query(query_3)

In [ ]:
result_3

## Question 4: SQL + pandas + plotting

<p>Expected result:</p>
<img src="data/plot_result.png">

In [ ]:
def plot(df : pd.DataFrame):
    #TODO: YOUR CODE GOES HERE
    # Make a copy so we don't modify the original result_3 DataFrame
    temp_df = df.copy()

    # Convert the tweetAt column to datetime (requirement #4)
    temp_df["TweetAt"] = pd.to_datetime(temp_df["TweetAt"], format="%d-%m-%Y")

    # Set the datetime column as the index
    temp_df.set_index('TweetAt', inplace=True)

    # Resample by day, aggregating by mean (requirement #2)
    daily_df = temp_df.resample('D').mean()

    # Plot the pos_rate time series using only pandas plot interface (requirement #3)
    daily_df['pos_rate'].plot()

In [ ]:
plot(result_3)

# PART 2: Object Oriented Programming + Data Wrangling

## Question 5

In [ ]:
import csv
from datetime import datetime
from math import sqrt

class Data:
    """
    This class holds the CSV data internally and provides:
      1) a mechanism to infer dtypes of each column,
      2) a method to describe() numerical columns (mean, std, min, max).
    """

    def __init__(self, header, rows):
        """
        :param header: list of column names
        :param rows: list of lists, where each sublist is a row of CSV data
        """
        self.header = header
        self.data = {}  # This dict will map {column_name: [col_values]}
        self.dtypes = {}  # Will store the inferred data types per column

        # Initialize self.data with an empty list for each column
        for col_name in self.header:
            self.data[col_name] = []

        # Populate self.data by columns
        for row in rows:
            for col_name, col_val in zip(self.header, row):
                self.data[col_name].append(col_val)

        # Infer data types after the data has been stored
        self.dtypes = self.infer_dtypes()
    
    def infer_dtypes(self) -> dict:
        """
        Infers the data types of each column distinguishing between date, integer, or float.
        If a column fails all numeric/date checks, it will be considered 'string'.

        Returns: dict of the form { column_name: "date"/"integer"/"float"/"string" }
        """
        inferred = {}

        for col_name, values in self.data.items():
            # We'll do integer -> float -> date -> string checks.

            # 1) Check integer possibility
            if self._all_integers(values):
                inferred[col_name] = "integer"
                continue

            # 2) Check float possibility
            if self._all_floats(values):
                inferred[col_name] = "float"
                continue

            # 3) Check date possibility (assuming "%d-%m-%Y")
            if self._all_dates(values, date_format="%d-%m-%Y"):
                inferred[col_name] = "date"
                continue

            # If none matched, store as string
            inferred[col_name] = "string"

        return inferred

    def describe(self) -> dict:
        """
        Returns basic statistics of each numerical column: mean, std, max, min.

        Example output:
            {
                "col_name": {
                    "mean": float,
                    "std": float,
                    "min": float,
                    "max": float
                },
                ...
            }
        """
        stats = {}

        for col_name, col_dtype in self.dtypes.items():
            if col_dtype in ("integer", "float"):
                # Gather non-NaN numeric values
                numeric_values = []
                for val in self.data[col_name]:
                    # We may have replaced errors with 'NaN' (string), so skip them
                    if val == "NaN":
                        continue
                    if isinstance(val, int) or isinstance(val, float):
                        numeric_values.append(val)
                    else:
                        # If stored as string but declared numeric, try to parse
                        try:
                            numeric_values.append(float(val))
                        except ValueError:
                            continue

                if len(numeric_values) == 0:
                    # If no valid numeric data, skip the stats
                    continue

                col_mean = sum(numeric_values) / len(numeric_values)
                # Calculate sample standard deviation
                variance = (
                    sum((x - col_mean)**2 for x in numeric_values) / (len(numeric_values) - 1)
                    if len(numeric_values) > 1 else 0.0
                )
                col_std = sqrt(variance)
                col_min = min(numeric_values)
                col_max = max(numeric_values)

                stats[col_name] = {
                    "mean": col_mean,
                    "std": col_std,
                    "min": col_min,
                    "max": col_max
                }
        return stats

    @staticmethod
    def _all_integers(values):
        """
        Return True if every non-"NaN" value can be parsed as integer.
        """
        for val in values:
            if val == "NaN":
                continue
            try:
                int(val)
            except ValueError:
                return False
        return True

    @staticmethod
    def _all_floats(values):
        """
        Return True if every non-"NaN" value can be parsed as float.
        """
        for val in values:
            if val == "NaN":
                continue
            try:
                float(val)
            except ValueError:
                return False
        return True

    @staticmethod
    def _all_dates(values, date_format="%Y-%m-%d"):
        """
        Return True if every non-"NaN" value can be parsed as a date
        using the given format.
        """
        for val in values:
            if val == "NaN":
                continue
            try:
                datetime.strptime(val, date_format)
            except ValueError:
                return False
        return True

class DataReader:
    """
    Responsible for reading a CSV file (with potential errors) into a Data object,
    replacing errors with 'NaN' if possible, or dropping rows ONLY if absolutely necessary.
    """

    def __init__(self):
        pass

    def read_csv(self, file_path: str) -> 'Data':
        rows = []
        header = []

        with open(file_path, mode="r", encoding="utf-8") as f:
            reader = csv.reader(f)
            first_line = True

            for row in reader:
                if first_line:
                    header = row
                    first_line = False
                    continue

                # Align to the length of the header
                if len(row) < len(header):
                    row += ["NaN"] * (len(header) - len(row))
                elif len(row) > len(header):
                    row = row[:len(header)]

                # Replace empty strings with 'NaN'
                cleaned_row = [col if col.strip() != "" else "NaN" for col in row]
                rows.append(cleaned_row)

        return Data(header, rows)


### How to use it:

In [ ]:
# ---------------------------------------------------
# Suppose you have the classes Data and DataReader
# either in the same Python file or properly imported:
# from my_data_lib import Data, DataReader
# ---------------------------------------------------

# 1) Instantiate the reader
reader = DataReader()

# 2) Read the CSV file into a Data object
data_object = reader.read_csv("example.csv")  # or malformed_dataset.csv

# 3) Look at the inferred data types
print("Inferred dtypes:", data_object.dtypes)

# 4) Print out basic statistics for numeric columns
stats = data_object.describe()
print("\nDescriptive statistics for numeric columns:")
for col_name, col_stats in stats.items():
    print(f"{col_name}: {col_stats}")
